In [20]:
#code adapted from https://nipunbatra.github.io/blog/ml/2020/03/29/param-learning.html
#and http://krasserm.github.io/2018/03/19/gaussian-processes/

#code is plotted with gaussian process utilities library found at: https://raw.githubusercontent.com/krasserm/bayesian-machine-learning/dev/gaussian-processes/gaussian_processes_util.py

#http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
        # 2.2, Algorithm 2.1.

In [21]:
import numpy as np
from gaussian_processes_util import plot_gp
from numpy.linalg import inv

def RBF(X1, X2, l=1.0, sigma_f=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def posterior(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    K = RBF(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s = RBF(X_train, X_s, l, sigma_f)
    K_ss = RBF(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
    K_inv = inv(K)

    mu_s = K_s.T.dot(K_inv).dot(Y_train)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
    
    return mu_s, cov_s


In [22]:
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize

def nll_fn(X_train, Y_train, sigma2, l2, noise):   
    Y_train = Y_train.ravel()
    K = RBF(X_train, X_train, l=l2, sigma_f=sigma2) + \
        noise**2 * np.eye(len(X_train))
    return 0.5 * np.log(det(K)) + \
            0.5 * Y_train.dot(inv(K).dot(Y_train)) + \
            0.5 * len(X_train) * np.log(2*np.pi)
    


In [23]:
import autograd.numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from autograd import elementwise_grad as egrad
from autograd import grad


def gradient_descent(X, Y, g, alpha, max_its, sigma, l):
    noise = 1.0
    gradient = grad(g, argnum=[0, 1, 2])
    cost_history = np.zeros(max_its)
    for iteration in range(max_its):
        cost_history[iteration] = g(X, Y, sigma, l, noise)
        del_sigma, del_l, del_noise = gradient(X, Y, sigma, l, noise)
        sigma = sigma - alpha*del_sigma
        l = l - alpha*del_l
        noise = noise - alpha*del_noise
    return cost_history, l, sigma, noise

In [25]:
from gaussian_processes_util import plot_gp_2D

noise_2D = 0.1

rx, ry = np.arange(-5, 5, 0.3), np.arange(-5, 5, 0.3)
gx, gy = np.meshgrid(rx, rx)

X_2D = np.c_[gx.ravel(), gy.ravel()]

X_2D_train = np.random.uniform(-4, 4, (100, 2))
Y_2D_train = np.sin(0.5 * np.linalg.norm(X_2D_train, axis=1)) + \
             noise_2D * np.random.randn(len(X_2D_train))

cost_history, l2, sigma, _ = gradient_descent(X_2D_train, Y_2D_train, g = nll_fn, alpha = 0.01, max_its = 350, sigma =2.0, l = 2.0)

mu_s, _ = posterior(X_2D, X_2D_train, Y_2D_train, l = l2, sigma_f = sigma)


TypeError: No loop matching the specified signature and casting was found for ufunc det